# Some Plots for the Thesis

In [ ]:
import os
os.environ["MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR"] = "False"
import mlflow 
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from helpers import get_transferability_data_from_mlflow, get_threshold_val, get_attacker_regret_errors
import pandas as pd
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [ ]:
load_dotenv()

In [ ]:
# Also create a column for the attacker_sig
# Helper function to safely get parameter values
def safe_get_param(row, param_name, default="None"):
    if param_name in row and pd.notna(row[param_name]):
        return str(row[param_name])
    return default

def format_str_to_same_length(inp1, inp2, inp3, padding1, padding2, padding3):
    len_1 = len(inp1)
    len_2 = len(inp2)
    len_3 = len(inp3)
    # pad all of them using _
    inp1 = inp1 + "_" * (padding1 - len_1)
    inp2 = inp2 + "_" * (padding2 - len_2)
    inp3 = inp3 + "_" * (padding3 - len_3)
    # return one string
    return f"{inp1}_{inp2}_{inp3}"

def insert_spacers(z, labels, group_keys):
    new_z = []
    new_labels = []
    prev_key = None

    group_id = 0
    for i, (row, label, key) in enumerate(zip(z, labels, group_keys)):
        if prev_key is not None and key != prev_key:
            new_z.append([np.nan] * z.shape[1])  # insert blank row
            new_labels.append(f"Group: {group_id+1}")                # blank label
            group_id += 1
        new_z.append(row)
        new_labels.append(label)
        prev_key = key

    return np.array(new_z), new_labels

def load_data(dataset_name):
    dataset_name = dataset_name + "_Models"
    # TODO: Adjust to your experiment id
    experiment_id = "228319214249946667"
    # Get all the models with img_size 12 -> get a list of run ids
    runs = mlflow.search_runs(experiment_ids=experiment_id)
    # Filter for finished
    runs = runs[runs["status"] == "FINISHED"]
    # Filter for dataset_name
    runs = runs[runs["params.attacked_models_experiment"] == dataset_name]
    # Assert that all models have been attacked
    runs["attacker_sig"] = runs.apply(lambda row: 
        row["params.attacker"] + 
        "_e" + safe_get_param(row, "params.epsilon") +
        "_a" + safe_get_param(row, "params.alpha") +
        "_m" + safe_get_param(row, "params.max_iter") +
        "_r" + safe_get_param(row, "params.restarts") +
        ("_s" + safe_get_param(row, "params.use_signed_grad") if "params.use_signed_grad" in row else ""), 
        axis=1)
    # In case of warcraft change the attacked_models_name
    if dataset_name == "Warcraft_Models":
        runs["params.attacked_models_name"] = runs["params.attacked_models_name"].apply(lambda x: x.replace("_regret", ""))
    runs["model_epsilon"] = runs["params.attacked_models_name"].astype(str) + "_eps=" + runs["params.epsilon"].astype(str)
    return runs

In [ ]:
# Load the kp data
kp_data = load_data("Knapsack")
spp_data = load_data("ShortestPath")
wcsp_data = load_data("Warcraft")

In [ ]:
model_name_dict = {
    'baseline_mse' : "PF",
    'baseline':  "PF",
    'SPO' : "SPO",
    'DBB' : "DBB",
    "IMLE" : "IMLE",
    'FenchelYoung' : "FY",
    "DCOL" : "QPTL",
    'IntOpt' : "IntOpt",
    'CachingPO_listwise' : "Listwise",
    'CachingPO_pairwise' : "Pairwise",
    'CachingPO_pairwise_diff' : "PairwiseDiff",
    'CachingPO_MAP_c' : "MAP",
}

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

# Define model order
model_order = ["PF", "SPO", "DBB", "IMLE", "FY", "QPTL", "IntOpt", "Listwise", "Pairwise", "PairwiseDiff", "MAP"]

def process_dataset(data, problem_name):
    """Process a dataset and return FRRE dictionaries and plot data"""
    frre_0025 = {}
    frre_005 = {}
    frre_01 = {}
    
    # First filter only for medium attack budget
    data = data[(data["params.max_iter"] == "100") | (data["params.max_iter"].isna()) | (data["params.max_iter"] == "50")]
    
    # For each of the datasets -> group by attacked_model_run_id and epsilon
    data_grouped = data.groupby(["params.attacked_models_run_id", "params.epsilon"])
    
    for (attacked_model_run_id, epsilon), group in data_grouped:
        modelname = model_name_dict[group["params.attacked_models_name"].values[0]]
        # Get the run id with the highest mean relative regret error
        idx_max = group["metrics.mean_rel_regret"].idxmax()
        attacker_run_id = group.loc[idx_max, "run_id"]
        attacker_name = group.loc[idx_max, "params.attacker"]
        
        # Now for the given attacker_run_id -> download the frre errors
        rre_clean, are_clean, rre_adv, are_adv, frre_adv = get_attacker_regret_errors(attacker_run_id, problem=problem_name)
        
        if epsilon == "0.025":
            assert attacked_model_run_id not in frre_0025, f"Duplicate entry for {attacked_model_run_id} with epsilon {epsilon}"
            frre_0025[attacked_model_run_id] = (f"{modelname}", frre_adv)
        elif epsilon == "0.05":
            assert attacked_model_run_id not in frre_005, f"Duplicate entry for {attacked_model_run_id} with epsilon {epsilon}"
            frre_005[attacked_model_run_id] = (f"{modelname}", frre_adv)
        elif epsilon == "0.1":
            assert attacked_model_run_id not in frre_01, f"Duplicate entry for {attacked_model_run_id} with epsilon {epsilon}"
            frre_01[attacked_model_run_id] = (f"{modelname}", frre_adv)
        else:
            raise ValueError(f"Unexpected epsilon value: {epsilon}")
    
    # Prepare data for plotting
    plot_data = []
    # Process each epsilon value - include all possible epsilon values
    epsilon_configs = [
        ("0.025", frre_0025, "ε = 0.025"),
        ("0.05", frre_005, "ε = 0.05"),  # Changed label to include (8)
        ("0.1", frre_01, "ε = 0.1"),
    ]
    
    for epsilon_val, epsilon_dict, epsilon_label in epsilon_configs:
        for model_id, (modelname, frre_values) in epsilon_dict.items():
            # Add each FRRE value as a separate row
            for frre_val in frre_values:
                # Fix epsilon=8 positioning to be same as 0.05 (second position)
                epsilon_numeric = 0.05 if epsilon_val == "8" else float(epsilon_val)
                plot_data.append({
                    'modelname': modelname,
                    'epsilon': epsilon_label,
                    'frre_value': frre_val,
                    'epsilon_numeric': epsilon_numeric
                })
    
    # Create DataFrame
    df_plot = pd.DataFrame(plot_data)
    if not df_plot.empty:
        # Sort by epsilon for consistent ordering
        df_plot = df_plot.sort_values('epsilon_numeric')
        # Create categorical column for model ordering
        df_plot['modelname'] = pd.Categorical(df_plot['modelname'], categories=model_order, ordered=True)
        df_plot = df_plot.sort_values(['epsilon_numeric', 'modelname'])
    
    return df_plot

# Process all datasets
kp_plot_data = process_dataset(kp_data, "Knapsack")
spp_plot_data = process_dataset(spp_data, "ShortestPath")

# Create subplots with minimal spacing
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=["Knapsack", "Shortest Path"],
    shared_xaxes=True,
    vertical_spacing=0.07  # Minimize space between plots
)

# Colors for epsilon values - epsilon=8 uses same color as epsilon=0.05
colors = {
    'ε = 0.025': '#1f77b4', 
    'ε = 0.05': '#ff7f0e',  # Same color for both 0.05 and 8
    'ε = 0.1': '#2ca02c'
}

# Add boxplots for each dataset
datasets = [
    (kp_plot_data, 1),
    (spp_plot_data, 2), 
]

# Keep track of which epsilon values we've added to legend
added_to_legend = set()

for df_plot, row in datasets:
    if not df_plot.empty:
        for epsilon in df_plot['epsilon'].unique():
            epsilon_data = df_plot[df_plot['epsilon'] == epsilon]
            
            for modelname in epsilon_data['modelname'].unique():
                model_data = epsilon_data[epsilon_data['modelname'] == modelname]
                
                # Only show in legend if not already added
                show_in_legend = epsilon not in added_to_legend
                if show_in_legend:
                    added_to_legend.add(epsilon)
                
                fig.add_trace(
                    go.Box(
                        y=model_data['frre_value'],
                        name=epsilon,
                        x=[str(modelname)] * len(model_data),  # Ensure string for categorical
                        marker_color=colors[epsilon],
                        legendgroup=epsilon,
                        showlegend=show_in_legend,
                        offsetgroup=epsilon,
                    ),
                    row=row, col=1
                )

# Update layout with legend at bottom
fig.update_annotations(font=dict(size=10))
fig.update_layout(
    boxmode='group',
    template="plotly_white",
    width=600,
    height=350,  # Reduced height since we minimized spacing
    showlegend=True,
    legend=dict(
        orientation="h",  # Horizontal legend
        yanchor="top",
        y=-0.15,  # Position below the plots
        xanchor="center",
        x=0.5,  # Center the legend
        title_text="Attack Magnitude",
        font=dict(size=10),  # Legend entries
    ),
    margin=dict(
        l=10,   # Left margin
        r=10,   # Right margin (reduced since legend is inside plot area)
        t=20,   # Top margin
        b=30    # Bottom margin (for x-axis label)
    ),
)
fig.update_xaxes(tickfont_size=10)
fig.update_yaxes(tickfont_size=10)

# Update axes - shared y-axis title on middle subplot only # change size to 10
fig.update_yaxes(title_text="FRRE")
fig.update_xaxes(title_text="Model", row=2)

# Set categorical x-axis ordering for all subplots
for row in [1, 2]:
    fig.update_xaxes(categoryorder='array', categoryarray=model_order, row=row)

fig.show()

In [ ]:
# TODO: Adjust this to your desired path
path = ""
fig.write_image(path, format="pdf", width=600, height=350)

In [ ]:
# Define model order
model_order = ["PF", "SPO", "DBB", "IMLE", "FY", "Listwise", "Pairwise", "PairwiseDiff", "MAP"]

def process_dataset(data, problem_name):
    """Process a dataset and return FRRE dictionaries and plot data"""
    frre_8 = {}
    
    # First filter only for medium attack budget
    data = data[(data["params.max_iter"] == "100") | (data["params.max_iter"].isna()) | (data["params.max_iter"] == "50")]
    
    # For each of the datasets -> group by attacked_model_run_id and epsilon
    data_grouped = data.groupby(["params.attacked_models_run_id", "params.epsilon"])
    
    for (attacked_model_run_id, epsilon), group in data_grouped:
        modelname = model_name_dict[group["params.attacked_models_name"].values[0]]
        # Get the run id with the highest mean relative regret error
        idx_max = group["metrics.mean_rel_regret"].idxmax()
        attacker_run_id = group.loc[idx_max, "run_id"]
        attacker_name = group.loc[idx_max, "params.attacker"]
        
        # Now for the given attacker_run_id -> download the frre errors
        rre_clean, are_clean, rre_adv, are_adv, frre_adv = get_attacker_regret_errors(attacker_run_id, problem=problem_name)
        
        if epsilon == "8":
            assert attacked_model_run_id not in frre_8, f"Duplicate entry for {attacked_model_run_id} with epsilon {epsilon}"
            frre_8[attacked_model_run_id] = (f"{modelname}", frre_adv)
        else:
            raise ValueError(f"Unexpected epsilon value: {epsilon}")
    
    # Prepare data for plotting
    plot_data = []
    # Process each epsilon value - include all possible epsilon values
    epsilon_configs = [
        ("8", frre_8, "ε = 8"),
    ]
    
    for epsilon_val, epsilon_dict, epsilon_label in epsilon_configs:
        for model_id, (modelname, frre_values) in epsilon_dict.items():
            # Add each FRRE value as a separate row
            for frre_val in frre_values:
                # Fix epsilon=8 positioning to be same as 0.05 (second position)
                epsilon_numeric = float(epsilon_val)
                plot_data.append({
                    'modelname': modelname,
                    'epsilon': epsilon_label,
                    'frre_value': frre_val,
                    'epsilon_numeric': epsilon_numeric
                })
    
    # Create DataFrame
    df_plot = pd.DataFrame(plot_data)
    if not df_plot.empty:
        # Sort by epsilon for consistent ordering
        df_plot = df_plot.sort_values('epsilon_numeric')
        # Create categorical column for model ordering
        df_plot['modelname'] = pd.Categorical(df_plot['modelname'], categories=model_order, ordered=True)
        df_plot = df_plot.sort_values(['epsilon_numeric', 'modelname'])
    
    return df_plot

# Process all datasets
wcsp_plot_data = process_dataset(wcsp_data, "Warcraft")

# Create subplots with minimal spacing
fig = make_subplots(
    rows=1, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.07  # Minimize space between plots
)

# Colors for epsilon values - epsilon=8 uses same color as epsilon=0.05
colors = {
    'ε = 8': '#ff7f0e',  # Same color for both 0.05 and 8
}

# Add boxplots for each dataset
datasets = [
    (wcsp_plot_data, 1)
]

# Keep track of which epsilon values we've added to legend
added_to_legend = set()

for df_plot, row in datasets:
    if not df_plot.empty:
        for epsilon in df_plot['epsilon'].unique():
            epsilon_data = df_plot[df_plot['epsilon'] == epsilon]
            for modelname in epsilon_data['modelname'].unique():
                model_data = epsilon_data[epsilon_data['modelname'] == modelname]
                # Only show in legend if not already added
                show_in_legend = epsilon not in added_to_legend
                if show_in_legend:
                    added_to_legend.add(epsilon)
                
                fig.add_trace(
                    go.Box(
                        y=model_data['frre_value'],
                        name=epsilon,
                        x=[str(modelname)] * len(model_data),  # Ensure string for categorical
                        marker_color=colors[epsilon],
                        legendgroup=epsilon,
                        showlegend=show_in_legend,
                        offsetgroup=epsilon,
                    ),
                    row=row, col=1
                )

# Update layout with legend at bottom
fig.update_annotations(font=dict(size=10))
fig.update_layout(
    boxmode='group',
    template="plotly_white",
    width=600,
    height=200,  # Reduced height since we minimized spacing
    showlegend=False,
    margin=dict(
        l=10,   # Left margin
        r=10,   # Right margin (reduced since legend is inside plot area)
        t=20,   # Top margin
        b=50    # Bottom margin (for x-axis label)
    ),
)
# For single plots do not show the title 
fig.update_xaxes(tickfont_size=10)
fig.update_yaxes(tickfont_size=10)

# Update axes - shared y-axis title on middle subplot only # change size to 10
fig.update_yaxes(title_text="FRRE", row=1)
fig.update_xaxes(title_text="Model", row=1)

# Set categorical x-axis ordering for all subplots
for row in [0]:
    fig.update_xaxes(categoryorder='array', categoryarray=model_order, row=row)

fig.show()

In [ ]:
# TODO: 
path = ""
fig.write_image("/storage/work/schaetz/plots/comparison_boxplots_wcsp.pdf", format="pdf", width=600, height=200)